In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plot
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
batch_size = 32
X_dim = 784
z_dim = 16
c_dim = 10
h_dim = 256

In [ ]:
mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)

In [ ]:
def plot_images(samples):
    fig = plot.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plot.subplot(gs[i])
        plot.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plot.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [ ]:
def xavier_init(size):
    input_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(input_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

In [ ]:
X = tf.placeholder(tf.float32, shape=[None, X_dim])

D_W1 = tf.Variable(xavier_init([X_dim, h_dim]))
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

D_W2 = tf.Variable(xavier_init([h_dim, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]


z = tf.placeholder(tf.float32, shape=[None, z_dim])
c = tf.placeholder(tf.float32, shape=[None, c_dim])

G_W1 = tf.Variable(xavier_init([z_dim + c_dim, h_dim // 2]))
G_b1 = tf.Variable(tf.zeros(shape=[h_dim // 2]))

G_W2 = tf.Variable(xavier_init([h_dim // 2, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

Q_W1 = tf.Variable(xavier_init([X_dim, h_dim // 2]))
Q_b1 = tf.Variable(tf.zeros(shape=[h_dim // 2]))

Q_W2 = tf.Variable(xavier_init([h_dim // 2, 10]))
Q_b2 = tf.Variable(tf.zeros(shape=[10]))

theta_Q = [Q_W1, Q_W2, Q_b1, Q_b2]

In [ ]:
def get_sample_z(size):
    return np.random.uniform(-1., 1., size=size)

In [ ]:
def get_sample_c(m):
    return np.random.multinomial(1, 10*[0.1], size=m)

In [ ]:
def generator(z, c):
    inputs = tf.concat(axis=1, values=[z, c])
    G_h1 = tf.nn.relu(tf.matmul(inputs, G_W1) + G_b1)
    G_out = tf.sigmoid(tf.matmul(G_h1, G_W2) + G_b2)
    return G_out

In [ ]:
def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_out = tf.matmul(D_h1, D_W2) + D_b2
    return D_out

In [ ]:
def Q(x):
    Q_h1 = tf.nn.relu(tf.matmul(x, Q_W1) + Q_b1)
    Q_out = tf.nn.softmax(tf.matmul(Q_h1, Q_W2) + Q_b2)
    return Q_out

In [ ]:
generator_sample = generator(z, c)
discriminator_real = discriminator(X)
discriminator_fake = discriminator(generator_sample)
q_c_x = Q(generator_sample)

In [ ]:
d_loss = -tf.reduce_mean(tf.log(discriminator_real + 1e-8) + tf.log(1 - discriminator_fake + 1e-8))
g_loss = -tf.reduce_mean(tf.log(discriminator_fake + 1e-8))
q_loss = tf.reduce_mean(-tf.reduce_sum(tf.log(q_c_x + 1e-8) * c, 1)) + \
            tf.reduce_mean(-tf.reduce_sum(tf.log(c + 1e-8) * c, 1))

In [ ]:
d_step = tf.train.AdamOptimizer(0.0001).minimize(d_loss, var_list=theta_D)
g_step = tf.train.AdamOptimizer(0.0001).minimize(g_loss, var_list=theta_G)
q_step = tf.train.AdamOptimizer(0.0001).minimize(q_loss, var_list=theta_G + theta_Q)

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
for i in range(100000):
    x_batch, _ = mnist.train.next_batch(batch_size)
    z_batch = get_sample_z((batch_size, z_dim))
    c_batch = get_sample_c(batch_size)
    
    _, d_loss_val = sess.run(
        [d_step, d_loss],
        feed_dict={X: x_batch, z: z_batch, c: c_batch}
    )

    _, g_loss_val = sess.run(
        [g_step, g_loss],
        feed_dict={z: z_batch, c: c_batch}
    )
    
    sess.run([q_step], feed_dict={z: z_batch, c: c_batch})

    if i % 100 == 0:
        print('Iteration: {} - Discriminator Loss: {:.4}, Generator Loss: {:.4}'
              .format(i, d_loss_val, g_loss_val))

        if i % 1000 == 0:
            samples = sess.run(generator_sample, feed_dict={z: get_sample_z(size=(16, z_dim)),
                                                            c: get_sample_c(16)})

            fig = plot_images(samples)
            plot.show()
            plot.close(fig)